# Homework 1

J Steven Raquel
PSTAT 134
Fri 9AM Section

## Simulating a confidence interval

For this assignment, I am going to create a simulation of a confidence interval of the Normal distribution, by simulating random observations from the distribution, computing the mean and variance estimates, and comparing them 
to the theoretical confidence results. 

More specifically, I am going to simulate from a Normal distribution with mean 100 and variance 20, and the confidence level I am going to use is 0.05.

In [1]:
import numpy as np
import scipy as sp
import scipy.stats